<a href="https://colab.research.google.com/github/lsmanoel/Keras_DeepLearning/blob/master/breast_cancer_diagnosis_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this exemple you will learn how to train a deep neural network to diagnose breast cancer. The packages required for this exemple are shown below:

In [0]:
import pandas as pd

from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

To load the database is shown below. The input_data_breast are the predictors and the output_data_breast are de real results of breast cancer diagnoses for each predictor.

In [0]:
previsores = pd.read_csv('entradas-breast.csv')
classe = pd.read_csv('saidas-breast.csv')

In [0]:
classificador = Sequential()
classificador.add(Dense(units = 8, activation = 'relu',
                        kernel_initializer = 'random_uniform', input_dim = 30))
classificador.add(Dropout(0.2))
classificador.add(Dense(units = 8, activation = 'relu',
                        kernel_initializer = 'normal'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units = 1, activation = 'sigmoid'))

#lr: learning rate ... decay: decaimento do learning rate
#clipvalue: limita os valores dos pesos
otimizador = keras.optimizers.Adam(lr = 0.001, decay = 0.0001, clipvalue = 0.5)
classificador.compile(optimizer = otimizador, loss = 'binary_crossentropy',
                      metrics = ['binary_accuracy'])

In [0]:
classificador.fit(previsores_treinamento, classe_treinamento,
                  batch_size = 10, epochs = 100)

In [0]:
pesos0 = classificador.layers[0].get_weights()
print(pesos0)
print(len(pesos0))
pesos1 = classificador.layers[1].get_weights()

In [0]:
previsoes = classificador.predict(previsores_teste)
previsoes = (previsoes > 0.5)
from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste, previsoes)
matriz = confusion_matrix(classe_teste, previsoes)

resultado = classificador.evaluate(previsores_teste, classe_teste)
print("Erro:%f Precisão:%f"% (resultado[0], resultado[1]))